# STEP 1

#### DEPENDENCIES

In [ ]:
import warnings

# Block all warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install transformers
!pip install bert-extractive-summarizer
!pip install torch

#### LIBRARY

In [3]:
from summarizer import Summarizer, TransformerSummarizer
from transformers import LongformerTokenizer
import os
import pandas as pd
import numpy as np

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### PREPROCESSING

In [5]:
data = "/content/drive/My Drive/cnn_dailymail"

In [6]:
# Load the CSV data as pandas dataframes
train_data = pd.read_csv(os.path.join(data, "train.csv"))
val_data = pd.read_csv(os.path.join(data, "validation.csv"))

In [ ]:
train_data.head()

In [8]:
train_data.columns

Index(['id', 'article', 'highlights'], dtype='object')

In [9]:
truncate = -1
df = pd.read_csv(os.path.join(data, "train.csv"))[:truncate]
df_val = pd.read_csv(os.path.join(data, "validation.csv"))[:truncate]
df_train = df[["article", "highlights"]]
df_train.columns = ["text", "summary"]
df_val = df_val[["article", "highlights"]]
df_val.columns = ["text", "summary"]

In [ ]:
df_train.tail()

# STEP 2

### TOKENIZER TEST

##### PHASE 1

In [11]:
# from transformers import BertTokenizer, GPT2Tokenizer, LongformerTokenizer
# from summarizer import Summarizer, TransformerSummarizer
# import nltk

# # Download the NLTK sentence tokenizer data (if not already downloaded)
# nltk.download('punkt')

# # Your input text
# body = '''
#         Eight people are dead following two shootings at shisha bars in the western German city of Hanau. At least five people were injured after gunmen opened fire at about 22:00 local time (21:00 GMT), police told the BBC. Police added that they are searching for the suspects, who fled the scene and are currently at large. The first shooting was at a bar in the city centre, while the second was in Hanau's Kesselstadt neighbourhood, according to local reports. Police officers and helicopters are patrolling both areas. An unknown number of gunmen killed three people at the first shisha bar, Midnight, before driving to the Arena Bar & Cafe and shooting dead another five victims, regional broadcaster Hessenschau reports. A dark-coloured vehicle was then seen leaving the scene.The motive for the attack is unclear, a police statement said. Can-Luca Frisenna, who works at a kiosk at the scene of one of the shootings said his father and brother were in the area when the attack took place. It's like being in a film, it's like a bad joke, that someone is playing a joke on us, he told Reuters.I can't grasp yet everything that has happened. My colleagues, all my colleagues, they are like my family - they can't understand it either. Hanau, in the state of Hessen, is about 25km (15 miles) east of Frankfurt. It comes four days after another shooting in Berlin, near a Turkish comedy show at the Tempodrom concert venue, which killed one person.
#         '''

# # BERT tokenizer for extractive part
# bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# # Tokenize the input text for extractive model
# inputs_extractive = bert_tokenizer(body, return_tensors="pt", max_length=512, truncation=True, padding=True)

# # Step 1: Extractive Summarization using BERT-based model
# extractive_model = Summarizer()
# extractive_summary = ''.join(extractive_model(body, min_length=60))

# # GPT2 tokenizer for abstractive part
# gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# gpt2_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# # Tokenize the extractive summary for abstractive model
# inputs_abstractive = gpt2_tokenizer(extractive_summary, return_tensors="pt", max_length=512, truncation=True, padding=True)

# # Longformer tokenizer for compressive part
# longformer_tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

# # Tokenize the abstractive summary for compressive model
# inputs_compressive = longformer_tokenizer(abstractive_summary, return_tensors="pt", max_length=4096, truncation=True, padding=True)

# # Step 3: Compressive Summarization using sentence scoring with the output of the abstractive model as input
# def compression_summary(text, num_sentences=3):
#     # Reusing the Longformer tokenizer for compression
#     tokenizer = longformer_tokenizer

#     # Tokenize the text into sentences using NLTK sentence tokenizer
#     sentences = nltk.sent_tokenize(text)

#     # Calculate scores for each sentence based on length and position in the text
#     scores = [len(sent.strip().split()) / len(text.split()) + i / len(sentences) for i, sent in enumerate(sentences)]

#     # Sort sentences based on scores and select the top sentences
#     selected_sentences = [sentences[i] for i in sorted(range(len(scores)), key=lambda k: scores[k], reverse=True)[:num_sentences]]

#     # Check if the last character of the generated summary is a period and remove it if present
#     if selected_sentences[-1].strip().endswith("."):
#         selected_sentences[-1] = selected_sentences[-1].strip()[:-1]

#     return " ".join(selected_sentences)

# compression_summary_text = compression_summary(abstractive_summary, num_sentences=3)
# final_summary = compression_summary_text
# # Final Summary: Combine the output from all three stages
# print(compression_summary_text)

##### PHASE 2

In [12]:
# import pandas as pd
# import os
# from transformers import BertTokenizer, GPT2Tokenizer, LongformerTokenizer
# from summarizer import Summarizer, TransformerSummarizer
# import nltk

# # Download the NLTK sentence tokenizer data (if not already downloaded)
# nltk.download('punkt')

# # Load the pre-trained tokenizers and models
# bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# gpt2_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# longformer_tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")
# extractive_model = Summarizer()

# # Define the compression summary function
# def compression_summary(text, num_sentences=3):
#     # Tokenize the text into sentences using NLTK sentence tokenizer
#     sentences = nltk.sent_tokenize(text)

#     # Calculate scores for each sentence based on length and position in the text
#     scores = [len(sent.strip().split()) / len(text.split()) + i / len(sentences) for i, sent in enumerate(sentences)]

#     # Sort sentences based on scores and select the top sentences
#     selected_sentences = [sentences[i] for i in sorted(range(len(scores)), key=lambda k: scores[k], reverse=True)[:num_sentences]]

#     # Check if the last character of the generated summary is a period and remove it if present
#     if selected_sentences[-1].strip().endswith("."):
#         selected_sentences[-1] = selected_sentences[-1].strip()[:-1]

#     return " ".join(selected_sentences)

# # Define the hybrid summarization pipeline
# def hybrid_summarization_pipeline(text):
#     # Step 1: Extractive Summarization using BERT-based model
#     inputs_extractive = bert_tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding=True)
#     extractive_summary = extractive_model(text, min_length=60)

#     # Step 2: Abstractive Summarization using GPT-2-based model
#     inputs_abstractive = gpt2_tokenizer(extractive_summary, return_tensors="pt", max_length=512, truncation=True, padding=True)

#     # Step 3: Compressive Summarization using sentence scoring with the output of the abstractive model as input
#     compression_summary_text = compression_summary(extractive_summary, num_sentences=3)
#     final_summary = compression_summary_text

#     return final_summary

# # Load the dataset and test the model using the hybrid pipeline
# data = "/content/drive/My Drive/cnn_dailymail"
# truncate = -1
# df = pd.read_csv(os.path.join(data, "train.csv"))[:truncate]

# for index, row in df.iterrows():
#     text = row['article']
#     summary = row['highlights']

#     # Test the hybrid pipeline on the text
#     generated_summary = hybrid_summarization_pipeline(text)

#     # Print the original summary and the generated summary
#     print("Original Summary:", summary)
#     print("Generated Summary:", generated_summary)
#     print("-" * 30)


### ADD DENSE LAYER EXTRACTIVE BERT STEP**
##### *FINETUNE*

In [ ]:
df_train['text']

In [ ]:
import pandas as pd
import os
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel, GPT2Tokenizer, LongformerTokenizer
from summarizer import Summarizer, TransformerSummarizer
import nltk

# Download the NLTK sentence tokenizer data (if not already downloaded)
nltk.download('punkt')

# Load the pre-trained tokenizers and models
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
longformer_tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")
extractive_model = Summarizer()

# Define the sentence scoring model on top of BERT
class SentenceScorer(nn.Module):
    def __init__(self, bert_model):
        super(SentenceScorer, self).__init__()
        self.bert = bert_model
        self.dense = nn.Linear(768, 1)  # 768 is the hidden size of BERT

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        scores = self.dense(pooled_output)
        return scores

# Instantiate the sentence scorer model
sentence_scorer = SentenceScorer(bert_model)

# Define the compression summary function using the sentence scorer
def compression_summary(text, num_sentences=3):
    # Tokenize the text into sentences using NLTK sentence tokenizer
    sentences = nltk.sent_tokenize(text)

    # Prepare input tensors for BERT
    inputs = bert_tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=512)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    # Get sentence scores using the sentence scorer model
    with torch.no_grad():
        scores = sentence_scorer(input_ids, attention_mask)

    # Convert scores to numpy array and normalize between 0 and 1
    scores = scores.squeeze().numpy()
    scores = (scores - scores.min()) / (scores.max() - scores.min())

    # Sort sentences based on scores and select the top sentences
    selected_sentences = [sentences[i] for i in scores.argsort()[::-1][:num_sentences]]

    # Check if the last character of the generated summary is a period and remove it if present
    if selected_sentences[-1].strip().endswith("."):
        selected_sentences[-1] = selected_sentences[-1].strip()[:-1]

    # Return the compressed summary and the sentence scores
    return " ".join(selected_sentences), scores

# Define the hybrid summarization pipeline
def hybrid_summarization_pipeline(text):
    # Step 1: Extractive Summarization using BERT-based model
    inputs_extractive = bert_tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding=True)
    extractive_summary = extractive_model(text, min_length=60)

    # Step 2: Abstractive Summarization using GPT-2-based model
    inputs_abstractive = gpt2_tokenizer(extractive_summary, return_tensors="pt", max_length=512, truncation=True, padding=True)

    # Step 3: Compressive Summarization using sentence scoring with the output of the abstractive model as input
    compression_summary_text, sentence_scores = compression_summary(extractive_summary, num_sentences=3)
    final_summary = compression_summary_text

    return final_summary, sentence_scores

# Load the dataset and test the model using the hybrid pipeline
data = "/content/drive/My Drive/cnn_dailymail"
truncate = -1
df = pd.read_csv(os.path.join(data, "train.csv"))[:truncate]

for index, row in df.iterrows():
    text = row['article']
    summary = row['highlights']

    # Test the hybrid pipeline on the text
    generated_summary, sentence_scores = hybrid_summarization_pipeline(text)

    # Print the original summary and the generated summary
    print("Original Summary:", summary)
    print("Generated Summary:", generated_summary)
    print("Sentence Scores:", sentence_scores)
    print("-" * 30)


### FINE TUNE ABSTRACTIVE GPT STEP

In [ ]:
import pandas as pd
import os
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel, GPT2Tokenizer, GPT2LMHeadModel, AdamW
from summarizer import Summarizer, TransformerSummarizer
import nltk
from tqdm import tqdm

# Download the NLTK sentence tokenizer data (if not already downloaded)
nltk.download('punkt')

# Load the pre-trained tokenizers and models
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
extractive_model = Summarizer()

# Define the sentence scoring model on top of BERT
class SentenceScorer(nn.Module):
    def __init__(self, bert_model):
        super(SentenceScorer, self).__init__()
        self.bert = bert_model
        self.dense = nn.Linear(768, 1)  # 768 is the hidden size of BERT

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        scores = self.dense(pooled_output)
        return scores

# Instantiate the sentence scorer model
sentence_scorer = SentenceScorer(bert_model)

# Define the compression summary function using the sentence scorer
def compression_summary(text, num_sentences=3):
    # Tokenize the text into sentences using NLTK sentence tokenizer
    sentences = nltk.sent_tokenize(text)

    # Prepare input tensors for BERT
    inputs = bert_tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=512)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    # Get sentence scores using the sentence scorer model
    with torch.no_grad():
        scores = sentence_scorer(input_ids, attention_mask)

    # Convert scores to numpy array and normalize between 0 and 1
    scores = scores.squeeze().numpy()
    scores = (scores - scores.min()) / (scores.max() - scores.min())

    # Sort sentences based on scores and select the top sentences
    selected_sentences = [sentences[i] for i in scores.argsort()[::-1][:num_sentences]]

    # Check if the last character of the generated summary is a period and remove it if present
    if selected_sentences[-1].strip().endswith("."):
        selected_sentences[-1] = selected_sentences[-1].strip()[:-1]

    # Return the compressed summary and the sentence scores
    return " ".join(selected_sentences), scores

# Define the hybrid summarization pipeline
def hybrid_summarization_pipeline(text):
    # Step 1: Extractive Summarization using BERT-based model
    inputs_extractive = bert_tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding=True)
    extractive_summary = extractive_model(text, min_length=60)

    # Step 2: Fine-tune GPT-2 on the extractive summary (target is the summary from the dataset)
    gpt2_model.train()
    inputs_gpt2 = gpt2_tokenizer(extractive_summary, return_tensors="pt", max_length=512, truncation=True, padding=True)
    labels = gpt2_tokenizer(extractive_summary, return_tensors="pt", max_length=512, truncation=True, padding=True)['input_ids']
    loss = gpt2_model(**inputs_gpt2, labels=labels).loss

    # Backpropagation and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Step 3: Compressive Summarization using sentence scoring with the output of the fine-tuned GPT-2 model as input
    compression_summary_text, sentence_scores = compression_summary(extractive_summary, num_sentences=3)
    final_summary = compression_summary_text

    return final_summary, sentence_scores

# Load the dataset and fine-tune the GPT-2 model on the extractive summaries
data = "/content/drive/My Drive/cnn_dailymail"
truncate = -1
df = pd.read_csv(os.path.join(data, "train.csv"))[:truncate]

# Set up fine-tuning parameters
num_epochs = 5
learning_rate = 2e-5
optimizer = AdamW(gpt2_model.parameters(), lr=learning_rate)

# Fine-tuning loop
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    for index, row in tqdm(df.iterrows(), total=len(df)):
        text = row['article']
        summary = row['highlights']

        # Test the hybrid pipeline on the text
        generated_summary, sentence_scores = hybrid_summarization_pipeline(text)

        # Print the original summary and the generated summary
        print("Original Summary:", summary)
        print("Generated Summary:", generated_summary)
        print("Sentence Scores:", sentence_scores)
        print("-" * 30)
